# Scraping
Primero comienzo por importar las librerías necesarias para realizar el scraping de la página web.

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

Ahora comienzo a realizar el scraping

In [ ]:
# Initialize the Selenium WebDriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open the Airbnb Barcelona page
url = "https://www.airbnb.com/s/Barcelona--Spain/homes"
driver.get(url)

# Let the page load for a few seconds to ensure that all content is loaded
time.sleep(5)  # You can adjust the sleep time or use WebDriver waits for better control

# Find all the listings on the page
listings = driver.find_elements(By.XPATH, '//div[contains(@class, "l1ovpqvx atm_1he2i46_1k8pnbi_10saat9 atm_yxpdqi_1pv6nv4_10saat9 atm_1a0hdzc_w1h1e8_10saat9 atm_2bu6ew_929bqk_10saat9 atm_12oyo1u_73u7pn_10saat9 atm_fiaz40_1etamxe_10saat9 bn2bl2p atm_5j_223wjw atm_9s_1ulexfb atm_e2_1osqo2v atm_fq_idpfg4 atm_mk_stnw88 atm_tk_idpfg4 atm_vy_1osqo2v atm_26_1j28jx2 atm_3f_glywfm atm_kd_glywfm atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_idpfg4_jo46a5 atm_3f_glywfm_1icshfk atm_kd_glywfm_19774hq atm_uc_aaiy6o_1w3cfyq_oggzyc atm_70_1b8lkes_1w3cfyq_oggzyc atm_uc_glywfm_1w3cfyq_pynvjw atm_uc_aaiy6o_pfnrn2_ivgyl9 atm_70_1b8lkes_pfnrn2_ivgyl9 atm_uc_glywfm_pfnrn2_61fwbc dir dir-ltr")]')  # Adjust XPATH
print(f"Number of listings: {len(listings)}")
properties = []

for listing in listings:
    if properties.length >= 100:
        try:
            # Extract the details for each listing
            name = listing.find_element(By.XPATH, './/a[contains(@class, "property_name_class")]').text
            address = listing.find_element(By.XPATH, './/span[contains(@class, "address_class")]').text
            price = listing.find_element(By.XPATH, './/span[contains(@class, "price_class")]').text
            rating = listing.find_element(By.XPATH, './/span[contains(@class, "rating_class")]').text
            num_rooms = listing.find_element(By.XPATH, './/span[contains(@class, "rooms_class")]').text
            
            # Append the data to the properties list
            properties.append({
                'Name': name,
                'Address': address,
                'Price per night': price,
                'Rating': rating,
                'Number of Rooms': num_rooms
            })
            
        except Exception as e:
            print(f"Error: {e}")
            continue
    else:
        break
# Create a pandas DataFrame
df = pd.DataFrame(properties)

# Save the DataFrame to a CSV file
df.to_csv('airbnb_barcelona.csv', index=False)
print("Data saved to airbnb_barcelona.csv")

# Close the browser
driver.quit()


Number of listings: 0
Data saved to airbnb_barcelona.csv
